\*\* *(under construction. If you're using Binder, please note that this tutorial is only fully functional in Firefox at the moment.)* \*\*

Adapted from [two](https://nbviewer.jupyter.org/github/marksubbarao/pyWWT_AAS225/blob/master/SgrStream.ipynb) [notebooks](https://nbviewer.jupyter.org/github/marksubbarao/pyWWT_AAS225/blob/master/CosmicFlows.ipynb) by Mark SubbaRao that use files from [here](http://faculty.virginia.edu/srm4n/Sgr/data.html) and visualize the tidal disruption of the Sagittarius Dwarf Galaxy by the Milky Way as described in [Law & Majewski 2010](https://iopscience.iop.org/article/10.1088/0004-637X/714/1/229).

In [2]:
import time

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, Column
from astropy.time import Time, TimeDelta

import numpy as np

from pywwt.jupyter import WWTJupyterWidget

In [3]:
wwt = WWTJupyterWidget()
wwt

*WWT works under HTTP, while Binder uses HTTPS. As such, in order to access all features of pywwt, Binder users will need to disable HTTPS for this session. Firefox users can do this with the following steps:*
- *Click the lock icon to the left of the address bar,*
- *Select the arrow next to “Connection,”*
- *Then, press the “Disable protection for now” button. (Protection is disabled for this page and only during this session, not for your whole browser. We are working on upgrading WWT so this step is no longer necessary.)*

## Simulating a galaxy merger and plotting stellar streams

(In this example, we a) simulate the position of the center of a dwarf galaxy during its near-merger with the Milky Way, and b) plot the positions of a snapshot of the resulting debris near the present day.)

**First, we retrieve a table with time and position data for the Sagittarius Dwarf Galaxy from a `pywwt`-affiliated GitHub repository. Then, we cut it down to only the columns needed for this example and assign them their proper units.**

In [8]:
orbit = Table.read('https://raw.githubusercontent.com/WorldWideTelescope/pywwt-notebooks/master/tutorials/data/SgrTriax_orbit.dat',
                   format='ascii.basic')
#orbit = Table.read('SgrTriax_orbit.dat', format='ascii.basic')

In [9]:
orbit.keep_columns(['time', 'ra', 'dec', 'dist'])
orbit['ra'].unit = u.deg
orbit['dec'].unit = u.deg
orbit['time'].unit = u.Gyr
orbit['dist'] *= 1000
orbit['dist'].unit = u.pc

**We'll also need to create a new column -- based on the original time column -- that converts and compresses our current gigayear times, which are too large for `pywwt` to simulate.** 

Instead, we'll convert them to Julian dates (which *are* readable by `pywwt`) and compress them so the action happens in a matter of seconds.

In [10]:
# translate times from gigayears, which are too large for pywwt, ...
# to days -- so 1 day in the viewer translates to 1 gigayear real-time

# T0-D0: find proper factor so that  1 gyr to translates to 10 seconds
base = Time('2019-01-01', format='iso').jd
orbit['wwt_times'] = base
orbit['wwt_times'] += orbit['time']

In [11]:
orbit

time,ra,dec,dist,wwt_times
Gyr,deg,deg,pc,
float64,float64,float64,float64,float64
-7.9961,29.7451,-8.51732,52743.6,2458476.5039
-7.9889,28.869,-8.92598,53198.1,2458476.5111
-7.9817,27.9937,-9.33262,53632.8,2458476.5183
-7.9744,27.1194,-9.73709,54049.6,2458476.5256
-7.9672,26.2453,-10.1392,54446.9,2458476.5328
-7.96,25.37,-10.5403,54826.799999999996,2458476.54
-7.9527,24.4935,-10.9395,55188.0,2458476.5473
-7.9455,23.6163,-11.3363,55531.5,2458476.5545


*(You can see how our table looks now by typing `orbit` in a new cell above this one and running it.)*

**Now, we can begin building the stellar streams table. The process is similar to the one for `orbit` -- importing the table, slim it down to the necessary columns, and give units to columns that need them.**

In [4]:
# import and manipulate a table with stellar stream data
stars = Table.read('https://raw.githubusercontent.com/WorldWideTelescope/pywwt-notebooks/master/tutorials/data/SgrTriax_DYN.dat',
                   format='ascii.basic')
#stars = Table.read('data/SgrTriax_DYN.dat', format='ascii.basic')

In [6]:
stars.keep_columns(["ra", "dec", "dist", "Pcol"])
stars['ra'].unit = u.deg
stars['dec'].unit = u.deg
stars['dist'] *= 1000
stars['dist'].unit = u.pc

**Next, we add a colormap column for pywwt to read in creating the data layer.** 

In Law & Majewski 2010, the authors color the tidal debris based on how many rotations it took before that patch of debris was stripped from its parent galaxy. We follow the same system, with the specific colors listed as comments below.

In [ ]:
stars['colormap'] = '#000000'
for i, val in enumerate(stars['Pcol']):
    if val == -1:
        stars['colormap'][i] = '#c0c0c0' # silver
    elif val == 0 or val == 1:
        stars['colormap'][i] = '#ffa500' # orange
    elif val == 2 or val == 3:
        stars['colormap'][i] = '#ff00ff' # magenta
    elif val == 4 or val == 5:
        stars['colormap'][i] = '#00ffff' # cyan
    elif val == 6 or val == 7:
        stars['colormap'][i] = '#228b22' # forest green

**Let's transition back to `pywwt` now that we have the tables we need for our data layers. Move the viewer down here and switch to the 3-D solar system view.**

In [ ]:
wwt.set_view('solar system')

**As indicated in this tutorial's title, we have two visualizations to make. First, we'll plot the center of mass of the Sagittarius Dwarf Galaxy in order to simulate its tidal disruption by the Milky Way.**

\*\* *(still under development, as we need working time series first)* \*\*

In [ ]:
orb_ly = wwt.layers.add_data_layer(table=orbit, frame='Sky',
                                   lon_att='ra', lat_att='dec',
                                   alt_att='dist', alt_type='distance',
                                   size_scale=20)

#wwt.play_time(...) # T0-D0: find proper rate at which to play time

Once the simulation has concluded, delete this layer and pause the passage of time to make way for the next visualization.

In [ ]:
wwt.pause_time()
orb_ly.remove()
# wwt.play_time(1) # ??

**In the second visualization, we plot a color-coded, present-day snapshot of the Sagittarius Dwarf's stellar streams.**

Again, we link attributes to the proper columns, equip the proper distance setting, and choose our desired point size. (Each attribute is also editable after the layer is created.)

For this one, we also set `far_side_visible` to `True` so we can see the debris even when it lies behind the Milky Way.

In [ ]:
dbr_ly = wwt.layers.add_data_layer(table=stars, frame='Sky',
                                   lon_att='ra', lat_att='dec',
                                   alt_att='dist', alt_type='distance',
                                   size_scale=40, far_side_visible=True)